# Useful Linux Utilities

This chapter will go through some common patterns in the shell and will include some useful Python commands that should enhance the ability to interact with a machine. 

## Mixing Python with Bash and ZSH


### Random Password Generator


In [4]:
import os, base64

print(base64.b64encode(os.urandom(64)).decode('utf-8'))

QmYhOnVXtPW5qMCuyEYE1vvxkvzEc0dxoZpCpCBkKV3Rs0x+tg/OXyfUK6Lv5jfxBIilpJqpP4Y+clAfP09pTQ==


Porting that to a shell function that can take an arbitrary length (useful when a site restricts length to a certain number) looks like this:
```shell
mpass() {
    if [ $1 ]; then
        length=$1
    else
        length=12
    fi
    _hash=`python3 -c "
import os,base64
exec('print(base64.b64encode(os.urandom(64))[:${length}].decode(\'utf-8\'))')
    "`
    echo $_hash | xclip -selection clipboard
    echo "new password copied to the system clipboard"
}
```
Now the mpass function defaults to generate 12-character passwords by slicing the output, and then sends the contents of the generated string to xclip so that it gets copied to the clipboard for easy pasting.

### Does my module exits?
```shell
try() {
    python -c "
exec('''
try:
    import ${1} as _
    print(_.__file__)
except Exception as e:
    print(e)
''')"
}
```

### Changing Directories to a Module’s Path
“Where does this module live?” is often asked when debugging libraries and dependencies, or even when poking around at the source of modules

In [6]:
# You can find out the path of a module if you import it and then use print
import os 
print(os)

<module 'os' from '/Users/jinhwan/opt/anaconda3/lib/python3.9/os.py'>


```shell
cdp() {
    MODULE_DIRECTORY=`python -c "
exec('''
try:
    import os.path as _, ${module}
    print(_.dirname(_.realpath(${module}.__file__)))
except Exception as e:
    print(e)
''')"`
    if  [[ -d $MODULE_DIRECTORY ]]; then
        cd $MODULE_DIRECTORY
    else
        echo "Module ${1} not found or is not importable: $MODULE_DIRECTORY"
    fi
}
```
